# GPU Training for Carbon/Water Footprint Model
Train the XGBoost multi-output model with Colab GPU acceleration.

**Features (v3.0):**
- **93 features** including formula features, fallback features, target encoding
- **Robustness training** with data augmentation (missing value simulation)
- Log-transformed targets for handling skewed distributions
- Automatic robustness testing at 0%, 20%, 40% missing data

**Training Modes:**
1. **Baseline only** - Maximum accuracy on complete data (R² = 0.9999)
2. **With augmentation** - Also trains robust model for missing data handling

**Setup:** Run cells in order. Use GPU runtime for faster training.

## 1a. Fresh Clone (ALWAYS run this first!)

This clones the code from GitHub. CSV files are uploaded separately (LFS bandwidth limit).

## 1b. Upload CSV Data Files

**GitHub LFS bandwidth exceeded!** Upload the CSV files manually.

Download these files from your local machine and upload them:
- `data/data_splitter/output/train.csv` (~180MB)
- `data/data_splitter/output/validate.csv` (~60MB)

In [ ]:
# ALWAYS run this cell first for a fresh start!
%cd /content

# Remove ANY old clone
!rm -rf /content/bulk_product_generator
!rm -rf /content/Avelero_HydroCarbon

# Clone code from GitHub (skip LFS - we'll get CSVs from Google Drive)
print("Cloning code from GitHub (without LFS)...")
!GIT_LFS_SKIP_SMUDGE=1 git clone https://github.com/Avelero/Avelero_HydroCarbon.git /content/bulk_product_generator

%cd /content/bulk_product_generator
print("\nLatest commit:")
!git log --oneline -3

# Mount Google Drive to get CSV files
print("\n" + "="*60)
print("MOUNTING GOOGLE DRIVE")
print("="*60)
from google.colab import drive
drive.mount('/content/drive')

# Copy CSV files from Google Drive
# INSTRUCTIONS: Upload these files to your Google Drive folder: MyDrive/data/
#   - train.csv (~269 MB)
#   - validate.csv (~90 MB)
#   - material_dataset_final.csv (~5 KB)
DRIVE_FOLDER = "/content/drive/MyDrive/data"  # Change this to your folder path

import os
import shutil

# Copy training/validation data
csv_dest = "/content/bulk_product_generator/data/data_splitter/output"
os.makedirs(csv_dest, exist_ok=True)

print(f"\nCopying CSV files from {DRIVE_FOLDER}...")
for f in ["train.csv", "validate.csv"]:
    src = f"{DRIVE_FOLDER}/{f}"
    dst = f"{csv_dest}/{f}"
    if os.path.exists(src):
        shutil.copy(src, dst)
        lines = sum(1 for _ in open(dst))
        print(f"  [OK] {f}: {lines:,} lines")
    else:
        print(f"  [ERROR] {src} not found!")

# Copy material dataset (for formula features)
material_dest = "/content/bulk_product_generator/data/data_calculations/input"
os.makedirs(material_dest, exist_ok=True)

material_src = f"{DRIVE_FOLDER}/material_dataset_final.csv"
material_dst = f"{material_dest}/material_dataset_final.csv"
if os.path.exists(material_src):
    shutil.copy(material_src, material_dst)
    lines = sum(1 for _ in open(material_dst))
    print(f"  [OK] material_dataset_final.csv: {lines:,} lines")
else:
    print(f"  [ERROR] {material_src} not found!")
    print(f"  Upload material_dataset_final.csv to: MyDrive/data/")

# Verify columns
print("\nVerifying train.csv columns...")
!head -1 /content/bulk_product_generator/data/data_splitter/output/train.csv | tr ',' '\n' | wc -l | xargs -I {} echo "Columns: {}"

# Install dependencies
%cd /content/bulk_product_generator/models
print("\nInstalling requirements...")
!pip install -q -r requirements.txt

print("\n[OK] Ready to train!")

In [ ]:
# ALWAYS run this cell first for a fresh start!
%cd /content

# Remove ANY old clone
!rm -rf /content/bulk_product_generator
!rm -rf /content/Avelero_HydroCarbon

# Clone fresh from GitHub (skip LFS to avoid bandwidth limits)
print("Cloning latest code from GitHub...")
!GIT_LFS_SKIP_SMUDGE=1 git clone https://github.com/Avelero/Avelero_HydroCarbon.git /content/bulk_product_generator

# Show latest commit to verify we have newest code
%cd /content/bulk_product_generator
print("\nLatest commit:")
!git log --oneline -3

# Install dependencies
%cd /content/bulk_product_generator/models
print("\nInstalling requirements...")
!pip install -q -r requirements.txt

print("\n[OK] Code cloned! Now run the next cell to upload CSV files.")

/content
Cloning into '/content/bulk_product_generator'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (418/418), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 418 (delta 168), reused 330 (delta 84), pack-reused 0 (from 0)
Receiving objects: 100% (418/418), 5.92 MiB | 12.08 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Filtering content: 100% (12/12), 1001.31 MiB | 72.36 MiB/s, done.
/content/bulk_product_generator/models


## 2. Check GPU

Verify GPU is available. Should show Tesla T4 or similar.

In [ ]:
# Check GPU availability
import torch
print(f" PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f" GPU: {torch.cuda.get_device_name(0)}")

print("\n GPU Details:")
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

Wed Dec  3 16:33:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 3. Train Model

**Options:**
- `QUICK_TEST = True` → 10K samples for quick testing (~30 sec)
- `TRAIN_ROBUST = True` → Train with data augmentation for robustness to missing data

**What TRAIN_ROBUST does:**
1. Trains baseline model on complete data (R² = 0.9999)
2. Trains robustness model with 15% missing weights, 25% missing distances, 10% missing materials
3. Tests both models with simulated missing data

This teaches the model to use **fallback features** (category averages, material intensity) when formula inputs are missing.

In [ ]:
# Configuration
QUICK_TEST = False  # Set to True for quick 10K sample test
TRAIN_ROBUST = True  # Set to True to also train with data augmentation for robustness

%cd /content/bulk_product_generator/models

if QUICK_TEST:
    print("⚡ QUICK TEST MODE (10K samples)")
    print("=" * 50)
    !python train_max_accuracy.py --sample-size 10000 --save-dir saved/gpu_test
else:
    print("🚀 FULL TRAINING MODE (all 676K samples)")
    print("=" * 50)
    
    if TRAIN_ROBUST:
        print("Training: Baseline + Robustness (with data augmentation)")
        print("Expected time: ~2 min baseline + ~2 min robustness + ~1 min testing")
        print()
        # --force-augmentation trains BOTH baseline and augmented models
        !python train_max_accuracy.py --save-dir saved/gpu_training --force-augmentation
    else:
        print("Training: Baseline only (complete data)")
        print("Expected time: ~2 min training + ~1 min robustness testing")
        print()
        !python train_max_accuracy.py --save-dir saved/gpu_training

/content/bulk_product_generator/models
Random seed set to 42
2025-12-03 16:33:47,183 - main - INFO - ================================================================================
2025-12-03 16:33:47,183 - main - INFO - MAXIMUM ACCURACY TRAINING STRATEGY
2025-12-03 16:33:47,183 - main - INFO - ================================================================================
2025-12-03 16:33:47,183 - main - INFO - Priority: Maximum accuracy (cost is not a constraint)
2025-12-03 16:33:47,183 - main - INFO - Dataset: Full 676K samples
2025-12-03 16:33:47,183 - main - INFO - 
2025-12-03 16:33:47,183 - main - INFO - 
2025-12-03 16:33:47,183 - main - INFO - PHASE 1: BASELINE TRAINING (Complete Data - Maximum Accuracy)
2025-12-03 16:33:47,183 - main - INFO - ================================================================================
2025-12-03 16:33:47,183 - main - INFO - 
Step 1/5: Loading data...
Loading training data from /content/bulk_product_generator/data/data_splitter/output/trai

: 

## 4. View Results

In [ ]:
# View training summary
print("📊 TRAINING RESULTS")
print("=" * 60)

# Show final metrics from log
print("\n🎯 BASELINE MODEL (complete data):")
!grep -E "(carbon_|water_).*R²" /content/bulk_product_generator/models/logs/training_max_accuracy.log | head -8

print("\n📈 ROBUSTNESS TEST RESULTS:")
!grep -E "Avg (MAE|R²)" /content/bulk_product_generator/models/logs/training_max_accuracy.log | tail -6

print("\n" + "=" * 60)
print("📁 Saved models:")
!ls -la /content/bulk_product_generator/models/saved/gpu_training/baseline/ 2>/dev/null || echo "  No baseline model yet"
!ls -la /content/bulk_product_generator/models/saved/gpu_training/robustness/ 2>/dev/null || echo "  No robustness model yet"

2025-12-03 16:09:55,470 - main - INFO - ================================================================================
2025-12-03 16:09:55,470 - main - INFO - MAXIMUM ACCURACY TRAINING STRATEGY
2025-12-03 16:09:55,470 - main - INFO - ================================================================================
2025-12-03 16:09:55,470 - main - INFO - Priority: Maximum accuracy (cost is not a constraint)
2025-12-03 16:09:55,470 - main - INFO - Dataset: Full 676K samples
2025-12-03 16:09:55,470 - main - INFO - 
2025-12-03 16:09:55,470 - main - INFO - 
2025-12-03 16:09:55,470 - main - INFO - PHASE 1: BASELINE TRAINING (Complete Data - Maximum Accuracy)
2025-12-03 16:09:55,471 - main - INFO - ================================================================================
2025-12-03 16:09:55,471 - main - INFO - 
Step 1/5: Loading data...
2025-12-03 16:10:01,539 - main - INFO - Step 2/5: Adding formula-based features...
2025-12-03 16:11:18,995 - main - INFO - Step 3/5: Preprocessing fea

## 📊 Quick Reference

### Expected Results:

**Baseline Model (complete data):**
| Target | R² | MAE |
|--------|-----|-----|
| carbon_material | 0.9999 | 0.04 kg |
| carbon_transport | 0.9998 | 0.001 kg |
| carbon_total | 0.9999 | 0.04 kg |
| water_total | 0.9998 | 115 L |

**Robustness (with augmented training):**
| Missing % | Baseline R² | Augmented R² |
|-----------|-------------|--------------|
| 0% | 0.9999 | ~0.999 |
| 20% | ~0.30 | **~0.85** |
| 40% | ~-0.40 | **~0.70** |

### Script Options:
```bash
python train_max_accuracy.py [OPTIONS]

--sample-size N       Use N samples (default: all 676K)
--save-dir DIR        Save directory
--force-augmentation  Train both baseline AND augmented models
--robustness-only     Skip baseline, train only augmented model
--skip-baseline       Skip baseline (use existing)
--seed N              Random seed (default: 42)
```

### Troubleshooting:
- **Old results?** → Re-run Cell 1 (Fresh Clone)
- **GPU not found?** → Runtime > Change runtime type > GPU
- **Out of memory?** → Use `--sample-size 100000`
- **Poor robustness?** → Set `TRAIN_ROBUST = True`

## 5. Download Model (Optional)

In [ ]:
# Download trained model and preprocessor
from google.colab import files
import shutil
import os

save_dir = "/content/bulk_product_generator/models/saved/gpu_training"

if os.path.exists(save_dir):
    # Create zip of saved model
    shutil.make_archive("/content/trained_model", 'zip', save_dir)
    print(" Created trained_model.zip")
    print(f"   Contents: {os.listdir(save_dir)}")
    
    # Download
    files.download("/content/trained_model.zip")
else:
    print("[ERROR] No saved model found. Run training first!")